In [ ]:
import os
import getpass
from glob import glob
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (RunnableParallel, RunnableLambda,
                                      ConfigurableFieldSpec, RunnablePassthrough)
from langchain.memory import ConversationBufferWindowMemory



#LLM
api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
chat = ChatOpenAI(model="gpt-3.5-turbo-1106", 
                  openai_api_key = api_key, temperature=0.2)

#Data(Big Bang Theory scripts files) Load
loader = DirectoryLoader("data", glob = "*.txt", show_progress=True)
all_script=loader.load()

#embedding
embed_model = OpenAIEmbeddings(openai_api_key = api_key,
                               model='text-embedding-3-small')
vecto_index = FAISS.from_documents(all_splits, embed_model)
vecto_index.save_local("data/script.json")
retriever = vecto_index.as_retriever(k=5)

#prompt_template
template_with_history = """
I want you act Dr. Sheldon Cooper from the TV show 'The Big Bang Theory'.
I want you to respond and answer like Sheldon Cooper using the tone, manner, and vocabulary Cooper would use.
You must know all of the knowlegde of Cooper.

If other's question is related with the TV show, adopt the part of the show script, with subtile revision to align with the question's intention.
Only reuse original lines if it improves the quality of the response.

Note that Cooper is a brilliant theoretical physicist with an IQ of 187. 
Cooper have a strict adherence to routine and find great difficulty in understanding sarcasm, irony, and social cues. 
Cooper is extremely logical, often to a fault, and have a tendency to be condescending when explaining things.

Answer the user's questions based on the below context. 
Real script of 'The Big Bang Theory' for the role are as follows:
###
{context}
###

{history}

user: {query}
Sheldon Cooper: 
"""


#retrieved_data merging
def merge_docs(retrieved_docs):
    return "###\n\n".join([d.page_content for d in retrieved_docs])

#Chain
prompt_history = ChatPromptTemplate.from_template(template_with_history)
memory = ConversationBufferWindowMemory(k=3, ai_prefix="Sheldon Cooper")
sheldon_chain = RunnableParallel({'context': retriever | merge_docs,
                          'query': RunnablePassthrough(),
                          'history':RunnableLambda(memory.load_memory_variables) | itemgetter('history')})\
                          | {'answer': prompt_history | chat | StrOutputParser(),
                             "context": itemgetter('context'),
                             "prompt": prompt_history}

